In [ ]:
import qp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cdshealpix.cdshealpix
import lsdb
from astropy.coordinates import SkyCoord
from astropy.io import fits
import matplotlib.pyplot as plt
import os

from astropy.wcs.utils import pixel_to_skycoord, skycoord_to_pixel
from cdshealpix import lonlat_to_healpix
from hats.inspection.visualize_catalog import plot_healpix_map
from hats.pixel_math import HealpixPixel
from hats.pixel_tree.moc_filter import filter_by_moc
from matplotlib.colors import LogNorm

In [ ]:
root_dir = "/sdf/data/rubin/shared/lsdb_commissioning/ecdfs-objects"
cat_path = os.path.join(root_dir, "galaxy_objects_curated_cds.pq")
fzboost_path = os.path.join(root_dir, "output_FZBoost.hdf5")
output_path = "/sdf/data/rubin/shared/lsdb_commissioning/sean_test/fzboost_curated_cds"

In [ ]:
object_cat = pd.read_parquet(cat_path)
object_cat

In [ ]:
fzboost_qp = qp.read(fzboost_path)

In [ ]:
fzboost_qp.npdf

In [ ]:
zmode = fzboost_qp.ancil['zmode']
zmedian = fzboost_qp.ancil['zmedian']
zmean = fzboost_qp.mean()

In [ ]:
for i in [1337, 1000, 3, 3413]:

    zgrid = np.linspace(0,3,301)
    pdf_example = fzboost_qp[i].pdf(zgrid)

    plt.plot(zgrid, pdf_example[0])
    plt.axvline(x=zmedian[i], color='r', linestyle='--', linewidth=1.5)
    plt.axvline(x=zmode[i], color='b', linestyle='--', linewidth=1.5)
    plt.axvline(x=zmean[i], color='g', linestyle='--', linewidth=1.5)
    plt.xlabel('redshift')
    plt.ylabel('PDF')
    plt.show()

In [ ]:
plt.hist(zmedian, bins = 30)
plt.show()

In [ ]:
import lsdb


In [ ]:
object_cat["zmedian"] = zmedian
object_cat["zmode"] = zmode
object_cat["zmean"] = zmean
object_cat

In [ ]:
cat = lsdb.from_dataframe(object_cat, ra_column="i_ra", dec_column="i_dec")

In [ ]:
cat

In [ ]:
cat.plot_pixels()

In [ ]:
cat.plot_coverage()

In [ ]:
# cat.to_hats(output_path, overwrite=True)

In [ ]:
fits_dir = "/sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_11/DM-49472/hips/deep/20250317T141618Z/deepCoadd_hpx/i/"
fits_files = [os.path.join(fits_dir, p) for p in os.listdir(fits_dir)]
fits_files[0]

In [ ]:
import re
hp_pixels = []
order = 11
pattern = r".*\/deepCoadd_hpx_i_([0-9]*)"

for f in fits_files:
    match = re.search(pattern, f)
    hp_pixels.append(HealpixPixel(order, int((match.group(1)))))

In [ ]:
hp_pixels

In [ ]:
from hats.pixel_tree.pixel_tree import PixelTree

tree = PixelTree.from_healpix(hp_pixels, order)

In [ ]:
from hats.inspection.visualize_catalog import plot_moc

plot_moc(tree.to_moc())

In [ ]:
cat = lsdb.read_hats("/sdf/data/rubin/shared/lsdb_commissioning/sean_test/fzboost_curated_cds")
cat

In [ ]:
filtered_tree = filter_by_moc(tree, cat.hc_structure.moc)
filtered_tree

In [ ]:
plot_moc(filtered_tree.to_moc())

In [ ]:
filtered_tree_pixels = filtered_tree.get_healpix_pixels()

In [ ]:
paths = [f"/sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_11/DM-49472/hips/deep/20250317T141618Z/deepCoadd_hpx/i/deepCoadd_hpx_i_{p.pixel}_LSSTComCam_runs_DRP_DP1_w_2025_11_DM-49472_hips_deep_20250317T141618Z.fits" for p in filtered_tree_pixels]
paths

In [ ]:
# for target in paths:
    # link_name = f"/sdf/home/s/smcgui/rubin-user/i_hips_in/{target.split("/")[-1]}"
    #
    # os.symlink(target, link_name)

In [ ]:
from lsdb.core.search.moc_search import MOCSearch

for pixel_num in range(1100,1200):

    p = filtered_tree_pixels[pixel_num]



    catalog_pixel = cat.search(MOCSearch(PixelTree.from_healpix([p], 11).to_moc())).compute()
    if len(catalog_pixel) > 10:
        print(p)
        print(pixel_num)
        break

catalog_pixel

In [ ]:
from astropy.wcs import WCS

with fits.open(paths[pixel_num]) as hdul:
    data = hdul[1].data
    header = hdul[1].header

wcs = WCS(header)
plt.figure(figsize=(12, 8))
plt.imshow(data, cmap='gray', vmin=5, vmax=30)
plt.colorbar()

In [ ]:
from astropy.wcs import WCS

with fits.open(paths[pixel_num]) as hdul:
    data = hdul[1].data
    header = hdul[1].header

wcs = WCS(header)
plt.figure(figsize=(12, 8))
plt.imshow(data, cmap='gray', vmin=5, vmax=30)
plt.colorbar()
x, y = skycoord_to_pixel(SkyCoord(catalog_pixel["i_ra"].to_numpy(), catalog_pixel["i_dec"].to_numpy(), unit="deg", frame="icrs"), wcs)
plt.scatter(x, y, c=catalog_pixel["zmode"].to_numpy(), marker='x', label="zmode", alpha=1)
plt.colorbar(label="zmode")

In [ ]:
w11 = lsdb.read_hats('/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_11/object_lc', margin_cache='/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_11/object_lc_5arcs')
w11

In [ ]:
res = cat.search(MOCSearch(PixelTree.from_healpix([p], 11).to_moc())).crossmatch(w11, suffixes=("", "_weekly11")).compute()
res

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def get_ellipse_outline(shape_xx, shape_yy, shape_xy, wcs, center=(0, 0)):
    if pd.isna(shape_xx) or pd.isna(shape_yy) or pd.isna(shape_xx):
        return None, None
    # Covariance matrix
    covariance_matrix = np.array([[shape_xx, shape_xy], [shape_xy, shape_yy]])
    # Eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    # Sorting eigenvalues and corresponding eigenvectors
    order = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[order]
    eigenvectors = eigenvectors[:, order]
    # Semi-major and semi-minor axes
    a = np.sqrt(eigenvalues[0]) # Semi-major axis
    b = np.sqrt(eigenvalues[1]) # Semi-minor axis
    # Orientation angle (in radians)
    theta = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])
    # Generate ellipse points
    t = np.linspace(0, 2 * np.pi, 100)
    ellipse_x = a * np.cos(t)
    ellipse_y = b * np.sin(t)
    # Rotation matrix
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    ellipse_points = np.dot(rotation_matrix, np.vstack((ellipse_x, ellipse_y)))
    # Plot the ellipse
    ra_points = ellipse_points[0] * 0.4 + center[0]
    dec_points = ellipse_points[1] * 0.4 + center[1]
    x, y = skycoord_to_pixel(SkyCoord(ra_points, dec_points, unit="arcsec", frame="icrs"), wcs)
    return x, y

def plot_ellipse(shape_xx, shape_yy, shape_xy, wcs, center=(0, 0)):
    x, y = get_ellipse_outline(shape_xx, shape_yy, shape_xy, wcs, center=center)
    if x is None:
        return
    plt.plot(x, y, label="Ellipse")
    plt.title("Ellipse from Shape Moments")
    plt.xlabel("X")
    plt.ylabel("Y")


In [ ]:
plot_ellipse(res.iloc[10]["shape_xx_weekly11"], res.iloc[10]["shape_yy_weekly11"], res.iloc[10]["shape_xy_weekly11"], wcs)

In [ ]:
import numpy as np
from astropy.wcs import WCS

wcs = WCS(header)
with fits.open(paths[pixel_num]) as hdul:
    data = hdul[1].data
    header = hdul[1].header

plt.figure(figsize=(12, 8))
plt.imshow(data, cmap='gray', vmin=5, vmax=30)
plt.colorbar()
for ii in range(len(res)):
    x, y = skycoord_to_pixel(SkyCoord(catalog_pixel["i_ra"].to_numpy()[ii], catalog_pixel["i_dec"].to_numpy()[ii], unit="deg", frame="icrs"), wcs)
    # plt.scatter(x, y, c=catalog_pixel["zmode"].to_numpy()[10], marker='x', label="zmode", alpha=1, s=0.1)
    plot_ellipse(res.iloc[ii]["shape_xx_weekly11"], res.iloc[ii]["shape_yy_weekly11"], res.iloc[ii]["shape_xy_weekly11"], wcs, center=(catalog_pixel["i_ra"].to_numpy()[ii] * 3600, catalog_pixel["i_dec"].to_numpy()[ii] * 3600))
# plt.colorbar(label="zmode")
# plt.xlim(200, 400)
# plt.ylim(400, 200)

In [ ]:
def get_ellipse_outline(shape_xx, shape_yy, shape_xy, wcs, center=(0, 0)):
    # Covariance matrix
    covariance_matrix = np.array([[shape_xx, shape_xy], [shape_xy, shape_yy]])
    # Eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    # Sorting eigenvalues and corresponding eigenvectors
    order = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[order]
    eigenvectors = eigenvectors[:, order]
    # Semi-major and semi-minor axes
    a = np.sqrt(eigenvalues[0]) # Semi-major axis
    b = np.sqrt(eigenvalues[1]) # Semi-minor axis
    # Orientation angle (in radians)
    theta = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])
    # Generate ellipse points
    t = np.linspace(0, 2 * np.pi, 100)
    ellipse_x = a * np.cos(t)
    ellipse_y = b * np.sin(t)
    # Rotation matrix
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    ellipse_points = np.dot(rotation_matrix, np.vstack((ellipse_x, ellipse_y)))
    # Plot the ellipse
    ra_points = ellipse_points[0] * 0.4 + center[0]
    dec_points = ellipse_points[1] * 0.4 + center[1]
    x, y = skycoord_to_pixel(SkyCoord(ra_points, dec_points, unit="arcsec", frame="icrs"), wcs)
    return x, y

def filter_image(imdata, objects, wcs, shape_xx_col="shape_xx_w11", shape_yy_col="shape_yy_w11", shape_xy_col="shape_xy_w11", ra_col="i_ra", dec_col="i_dec"):
    pixels_to_keep = []
    for i in range(len(objects)):
        if pd.isna(objects[shape_xx_col].iloc[i]) or pd.isna(objects[shape_yy_col].iloc[i]) or pd.isna(objects[shape_xy_col].iloc[i]):
            break
        x, y = get_ellipse_outline(objects[shape_xx_col].iloc[i], objects[shape_yy_col].iloc[i], objects[shape_xy_col].iloc[i], wcs, center=(objects[ra_col].to_numpy()[i] * 3600, objects[dec_col].to_numpy()[i] * 3600))
        all_pixels = np.vstack([x.astype(int), y.astype(int)]).T
        pixels = np.unique(all_pixels, axis=0)
        for i in range(len(pixels)-1):
            p1 = pixels[i]
            p2 = pixels[i+1]
            pixels_to_keep.append(p1)
            if p1[0] == p2[0]:
                if p1[1] + 1 < p2[1]:
                    for yi in range(p1[1] + 1, p2[1]):
                        pixels_to_keep.append(np.array([p1[0], yi]))
    pixels_to_keep = np.array(pixels_to_keep)
    filtered_im = np.zeros_like(imdata)
    if len(pixels_to_keep) > 0:
        filtered_im[pixels_to_keep.T[1].clip(max=filtered_im.shape[0]-1), pixels_to_keep.T[0].clip(max=filtered_im.shape[1]-1)] = imdata[pixels_to_keep.T[1].clip(max=filtered_im.shape[0]-1), pixels_to_keep.T[0].clip(max=filtered_im.shape[1]-1)]
    return filtered_im

In [ ]:
res

In [ ]:
import numpy as np
from astropy.wcs import WCS

with fits.open(paths[pixel_num]) as hdul:
    data = hdul[1].data
    header = hdul[1].header
    wcs = WCS(header)

modified_image = filter_image(data, res, wcs, shape_xx_col="shape_xx_weekly11",shape_yy_col="shape_yy_weekly11", shape_xy_col="shape_xy_weekly11" )


plt.figure(figsize=(12, 8))
plt.imshow(modified_image, cmap='gray', vmin=5, vmax=30)
plt.colorbar()

In [ ]:
fits_dir = "/sdf/home/s/smcgui/rubin-user/i_hips_in"
fits_files = [os.path.join(fits_dir, p) for p in os.listdir(fits_dir)]
fits_files

In [ ]:
import re
fits_files_hp_pixels = []
order = 11
pattern = r".*\/deepCoadd_hpx_i_([0-9]*)"

for f in fits_files:
    match = re.search(pattern, f)
    fits_files_hp_pixels.append(HealpixPixel(order, int((match.group(1)))))
fits_files_hp_pixels

In [ ]:
import dask.dataframe as dd

paths_ddf = dd.from_map(lambda f: pd.DataFrame.from_dict({"filepath": [f]}), fits_files, meta=pd.DataFrame({"filepath": pd.Series([], dtype="string")}))

In [ ]:
ddf_pixel_map = {p: i for i, p in enumerate(fits_files_hp_pixels)}
ddf_pixel_map

In [ ]:
from hats.catalog import TableProperties

map_cat_props = TableProperties(catalog_name="fits_paths_map_cat", catalog_type="map", total_rows=len(fits_files))
map_cat_props

In [ ]:
import hats

map_catalog_hc_structure = hats.catalog.MapCatalog(map_cat_props, fits_files_hp_pixels)
map_catalog_hc_structure

In [ ]:
from lsdb.catalog import MapCatalog

fits_paths_cat = MapCatalog(paths_ddf, ddf_pixel_map, map_catalog_hc_structure)
fits_paths_cat

In [ ]:
fzboost_cat = lsdb.read_hats("/sdf/data/rubin/shared/lsdb_commissioning/sean_test/fzboost_curated_cds")
fzboost_cat

In [ ]:
w11 = lsdb.read_hats('/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_11/object_lc', margin_cache='/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_11/object_lc_5arcs')
w11

In [ ]:
rubin = fzboost_cat.crossmatch(w11, suffixes=("", "_w11"))
rubin

In [ ]:
def get_ellipse_outline(shape_xx, shape_yy, shape_xy, wcs, center=(0, 0)):
    # Covariance matrix
    covariance_matrix = np.array([[shape_xx, shape_xy], [shape_xy, shape_yy]])
    # Eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    # Sorting eigenvalues and corresponding eigenvectors
    order = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[order]
    eigenvectors = eigenvectors[:, order]
    # Semi-major and semi-minor axes
    a = np.sqrt(eigenvalues[0]) # Semi-major axis
    b = np.sqrt(eigenvalues[1]) # Semi-minor axis
    # Orientation angle (in radians)
    theta = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])
    # Generate ellipse points
    t = np.linspace(0, 2 * np.pi, 100)
    ellipse_x = a * np.cos(t)
    ellipse_y = b * np.sin(t)
    # Rotation matrix
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    ellipse_points = np.dot(rotation_matrix, np.vstack((ellipse_x, ellipse_y)))
    # Plot the ellipse
    ra_points = ellipse_points[0] * 0.4 + center[0]
    dec_points = ellipse_points[1] * 0.4 + center[1]
    x, y = skycoord_to_pixel(SkyCoord(ra_points, dec_points, unit="arcsec", frame="icrs"), wcs)
    return x, y

def filter_image(imdata, objects, wcs, shape_xx_col="shape_xx_w11", shape_yy_col="shape_yy_w11", shape_xy_col="shape_xy_w11", ra_col="i_ra", dec_col="i_dec"):
    pixels_to_keep = []
    for i in range(len(objects)):
        if pd.isna(objects[shape_xx_col].iloc[i]) or pd.isna(objects[shape_yy_col].iloc[i]) or pd.isna(objects[shape_xy_col].iloc[i]):
            break
        x, y = get_ellipse_outline(objects[shape_xx_col].iloc[i], objects[shape_yy_col].iloc[i], objects[shape_xy_col].iloc[i], wcs, center=(objects[ra_col].to_numpy()[i] * 3600, objects[dec_col].to_numpy()[i] * 3600))
        all_pixels = np.vstack([x.astype(int), y.astype(int)]).T
        pixels = np.unique(all_pixels, axis=0)
        for i in range(len(pixels)-1):
            p1 = pixels[i]
            p2 = pixels[i+1]
            pixels_to_keep.append(p1)
            if p1[0] == p2[0]:
                if p1[1] + 1 < p2[1]:
                    for yi in range(p1[1] + 1, p2[1]):
                        pixels_to_keep.append(np.array([p1[0], yi]))
    pixels_to_keep = np.array(pixels_to_keep)
    filtered_im = np.zeros_like(imdata)
    if len(pixels_to_keep) > 0:
        filtered_im[pixels_to_keep.T[1].clip(max=filtered_im.shape[0]-1), pixels_to_keep.T[0].clip(max=filtered_im.shape[1]-1)] = imdata[pixels_to_keep.T[1].clip(max=filtered_im.shape[0]-1), pixels_to_keep.T[0].clip(max=filtered_im.shape[1]-1)]
    return filtered_im

In [ ]:
def split_image(partition, fits_path_df, catalog_pixel, map_pixel, zbins, out_base_paths):
    file_path = fits_path_df["filepath"].iloc[0]
    with fits.open(file_path) as hdul:
        out_path = os.path.join(out_base_paths[0], file_path.split("/")[-1])
        data = hdul[1].data
        header = hdul[1].header
        wcs = WCS(header)
        for bin, out_base_path in zip(zbins, out_base_paths):
            if len(bin) == 2:
                queried_result = partition.query(f"zmode >= {bin[0]} and zmode < {bin[1]}")
                out_data = filter_image(data, queried_result, wcs)
            elif len(bin) == 1:
                queried_result = partition.query(f"zmode >= {bin[0]}")
                out_data = filter_image(data, queried_result, wcs)
            out_path = os.path.join(out_base_path, file_path.split("/")[-1])
            hdu = fits.PrimaryHDU(out_data, header=header)
            out_hdul = fits.HDUList([hdu])
            out_hdul.writeto(out_path)
    return pd.DataFrame.from_dict({"lenpart": [len(partition)]})

In [ ]:
zbins = [[0, 0.1], [0.1, 0.3], [0.3, 0.7], [0.7, 1.5], [1.5]]
# zbins = [["initial"]]
out_paths = [f"/sdf/home/s/smcgui/rubin-user/filtered_hips_i/bin{i[0]}/" for i in zbins]
# for path in out_paths:
#     os.makedirs(path, exist_ok=False)
out = rubin.merge_map(fits_paths_cat, split_image, zbins, out_paths, meta=pd.DataFrame({"lenpart": pd.Series([], dtype="int")}))
out

In [ ]:
last_file = f"{out_paths[-3]}/deepCoadd_hpx_i_36796142_LSSTComCam_runs_DRP_DP1_w_2025_11_DM-49472_hips_deep_20250317T141618Z.fits"

In [ ]:
with fits.open(last_file) as hdul:
    test_data = hdul[0].data
    test_header = hdul[0].header

test_wcs = WCS(test_header)
plt.figure(figsize=(12, 8))
plt.imshow(test_data, cmap='gray', vmin=5, vmax=30)
plt.colorbar()
plt.show()

In [ ]:
import os
import shutil
from PIL import Image

def find_png_files(root_dir):
    """Find all PNG files in Norder directories and group them by pixel tile name."""
    png_files = {}

    for dirpath, _, filenames in os.walk(root_dir):
        if "Norder" in dirpath:
            for file in filenames:
                if file.endswith(".png"):
                    tile_name = os.path.basename(file)
                    png_files[tile_name] = os.path.join(dirpath, file)

    return png_files

def merge_rgb_images(red_dir, green_dir, blue_dir, output_dir):
    """Merge red, green, and blue PNGs into an RGB image and save to output_dir."""
    # os.makedirs(output_dir, exist_ok=True)

    red_files = find_png_files(red_dir)
    green_files = find_png_files(green_dir)
    blue_files = find_png_files(blue_dir)

    all_tiles = set(red_files.keys()) | set(green_files.keys()) | set(blue_files.keys())

    for tile in all_tiles:
        red_path = red_files.get(tile, {})
        green_path = green_files.get(tile, {})
        blue_path = blue_files.get(tile, {})

        red_img = Image.open(red_path).convert("L") if red_path else None
        green_img = Image.open(green_path).convert("L") if green_path else None
        blue_img = Image.open(blue_path).convert("L") if blue_path else None

        # Determine image size (use first available image size)
        img_size = red_img.size if red_img else (green_img.size if green_img else blue_img.size)

        # Create black images for missing channels
        if not red_img:
            continue
        if not green_img:
            continue
        if not blue_img:
            continue

        # Merge into RGB
        rgb_image = Image.merge("RGB", (red_img, green_img, blue_img))

        return rgb_image

        # Recreate directory structure in output folder
        # original_path = os.path.dirname(red_path)
        #
        # relative_path = os.path.relpath(original_path, red_dir)
        # save_path = os.path.join(output_dir, relative_path)
        # os.makedirs(save_path, exist_ok=True)

        # rgb_image.save(os.path.join(save_path, tile))
        # print(f"Saved: {os.path.join(save_path, tile)}")

In [ ]:
red_hips_dir = f"/sdf/home/s/smcgui/rubin-user/i_hips"
green_hips_dir = f"/sdf/home/s/smcgui/rubin-user/r_hips"
blue_hips_dir = f"/sdf/home/s/smcgui/rubin-user/g_hips"

In [ ]:
rgb_im = merge_rgb_images(red_hips_dir, green_hips_dir, blue_hips_dir, "")
rgb_im